In [1]:
#===从GCS加载数据

project_id = 'amazing-firefly-153908'
bucket_name = 'nlp_final'

# 获取授权
from google.colab import auth
auth.authenticate_user()

#从GCS下载数据至tmp
from googleapiclient.discovery import build
gcs_service = build('storage', 'v1')
from apiclient.http import MediaIoBaseDownload

W0822 09:30:46.893904 140678760236928 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0822 09:30:48.088112 140678760236928 __init__.py:44] file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py

In [0]:
def download_from_GCS(file_name):
  with open(file_name, 'wb') as f:
    request = gcs_service.objects().get_media(bucket=bucket_name,
                                              object=file_name)
    media = MediaIoBaseDownload(f, request)

    done = False
    while not done:
      # _ is a placeholder for a progress object that we ignore.
      # (Our file is small, so we skip reporting progress.)
      _, done = media.next_chunk()
  print(file_name + ' downloaded')

In [0]:
def get_time():
  seconds = time.time()
  named_tuple = time.gmtime(seconds) # get struct_time
  hour = named_tuple.tm_hour + 8
  if hour > 24:
    hour = hour - 24
  time_str = str(hour) + ':' + str(named_tuple.tm_min) + ':' + str(named_tuple.tm_sec)
  return time_str

In [3]:
download_from_GCS('train.csv')
# download_from_GCS('test.csv')

train.csv downloaded


In [0]:
import numpy as np
import pandas as pd
import time
from tqdm.auto import tqdm
tqdm.pandas()

In [0]:
train_data = pd.read_csv('train.csv')
# test_data = pd.read_csv('test.csv')

In [0]:
# Install the latest Tensorflow version.
!pip3 install --quiet "tensorflow>=1.7"
# Install TF-Hub.
!pip3 install --quiet tensorflow-hub
!pip3 install --quiet seaborn

In [0]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import os
import re
import seaborn as sns

In [0]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

In [9]:
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)

# Compute a representation for each message, showing various lengths supported.
word = "Elephant"
sentence = "I am a sentence for which I would like to get its embedding."
paragraph = (
    "Universal Sentence Encoder embeddings also support short paragraphs. "
    "There is no hard limit on how long the paragraph is. Roughly, the longer "
    "the more 'diluted' the embedding will be.")
messages = [word, sentence, paragraph]

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings = session.run(embed(messages))

  for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
    print("Message: {}".format(messages[i]))
    print("Embedding size: {}".format(len(message_embedding)))
    message_embedding_snippet = ", ".join(
        (str(x) for x in message_embedding[:3]))
    print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

Message: Elephant
Embedding size: 512
Embedding: [-0.01698727160692215, -0.00894981063902378, -0.007062731776386499, ...]

Message: I am a sentence for which I would like to get its embedding.
Embedding size: 512
Embedding: [0.035313352942466736, -0.025384269654750824, -0.00788002647459507, ...]

Message: Universal Sentence Encoder embeddings also support short paragraphs. There is no hard limit on how long the paragraph is. Roughly, the longer the more 'diluted' the embedding will be.
Embedding size: 512
Embedding: [0.01879092864692211, 0.04536517709493637, -0.020010894164443016, ...]



In [0]:
# train_data['question1'][0]
start = time.time()
with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  q1_embeddings = session.run(embed([train_data['question1'][0]]))
end = time.time()
print('It took {} s to embedding 1 sentences'.format(end - start))
print(q1_embeddings.shape)

It took 10.407913208007812 s to embedding 1 sentences
(1, 512)


In [0]:
# train_data['question1'][0]
start = time.time()
with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  qs_embeddings = session.run(embed([train_data['question1'][0],train_data['question2'][0]]))
end = time.time()
print('It took {} s to embedding 2 sentences'.format(end - start))
print(qs_embeddings.shape)

It took 11.035587549209595 s to embedding 2 sentences
(2, 512)


In [0]:
# type(train_data['question1'][0:10].tolist())

start = time.time()
with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  q10_embeddings = session.run(embed(train_data['question1'][0:10].tolist()))
end = time.time()
print('It took {} s to embedding 10 sentences'.format(end - start))
print(q10_embeddings.shape)

# type(train_data['question1'][0:10].tolist())

start = time.time()
with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  q20_embeddings = session.run(embed(train_data['question2'][0:10].tolist()))
end = time.time()
print('It took {} s to embedding 10 sentences'.format(end - start))
print(q20_embeddings.shape)

It took 9.811806201934814 s to embedding 10 sentences
(10, 512)
It took 10.447726964950562 s to embedding 10 sentences
(10, 512)


In [0]:
data = pd.DataFrame()
data['cosine_glv'] = [cosine(x, y) for (x, y) in zip(np.nan_to_num(q10_embeddings),np.nan_to_num(q20_embeddings))]

In [0]:
data

,cosine_glv
0,0.036198
1,0.262635
2,0.104333
3,0.766393
4,0.474255
5,0.112613
6,0.980641
7,0.047138
8,0.210263
9,0.250988


In [0]:
start = time.time()

L2 = np.sqrt(np.square(q10_embeddings - q20_embeddings ).sum(axis=1))

end = time.time()
print('It took {} s to compute L2 for 10 sentences'.format(end - start))

print(L2.shape)

It took 0.0007028579711914062 s to compute L2 for 10 sentences
(10,)


In [0]:
start = time.time()
with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  q100_embeddings = session.run(embed(train_data['question1'][0:100].tolist()))
end = time.time()
print('It took {} s to embedding 100 sentences'.format(end - start))
print(q100_embeddings.shape)

start = time.time()
with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  q200_embeddings = session.run(embed(train_data['question2'][0:100].tolist()))
end = time.time()
print('It took {} s to embedding 100 sentences'.format(end - start))
print(q200_embeddings.shape)

start = time.time()

L2 = np.sqrt(np.square(q100_embeddings - q200_embeddings ).sum(axis=1))

end = time.time()
print('It took {} s to compute L2 for 100 sentences'.format(end - start))

print(L2.shape)

It took 13.989994287490845 s to embedding 100 sentences
(100, 512)
It took 13.993663787841797 s to embedding 100 sentences
(100, 512)
It took 0.0006272792816162109 s to compute L2 for 100 sentences
(100,)


In [0]:
print(L2[0:10])

[0.26906368 0.7247549  0.45679983 1.2380573  0.97391504 0.4745791
 1.4004576  0.3070439  0.6484791  0.70850205]


In [0]:
start = time.time()
with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  q1000_embeddings = session.run(embed(train_data['question1'][0:1000].tolist()))
end = time.time()
print('It took {} s to embedding 1000 sentences'.format(end - start))
print(q1000_embeddings.shape)

start = time.time()
with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  q2000_embeddings = session.run(embed(train_data['question2'][0:1000].tolist()))
end = time.time()
print('It took {} s to embedding 1000 sentences'.format(end - start))
print(q2000_embeddings.shape)

start = time.time()

L2 = np.sqrt(np.square(q1000_embeddings - q2000_embeddings ).sum(axis=1))

end = time.time()
print('It took {} s to compute L2 for 1000 sentences'.format(end - start))

print(L2.shape)

It took 16.11455798149109 s to embedding 1000 sentences
(1000, 512)
It took 16.20926022529602 s to embedding 1000 sentences
(1000, 512)
It took 0.002185821533203125 s to compute L2 for 1000 sentences
(1000,)


In [0]:
start = time.time()
with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  q1000_embeddings = session.run(embed(train_data['question1'][0:1000].tolist()))
end = time.time()
print('It took {} s to embedding 1000 sentences'.format(end - start))
print(q1000_embeddings.shape)

start = time.time()
with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  q2000_embeddings = session.run(embed(train_data['question2'][0:1000].tolist()))
end = time.time()
print('It took {} s to embedding 1000 sentences'.format(end - start))
print(q2000_embeddings.shape)

start = time.time()

L2 = np.sqrt(np.square(q1000_embeddings - q2000_embeddings ).sum(axis=1))

end = time.time()
print('It took {} s to compute L2 for 1000 sentences'.format(end - start))

print(L2.shape)

In [0]:
from scipy.stats import skew, kurtosis
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis


In [0]:
train_data = train_data.fillna(" ")

batch_size = 300000
batch_num = train_data.shape[0]//batch_size

ft_train = pd.DataFrame({'encode_cos':0,'encode_L1':0,'encode_L2':0,
                         'canberra_encode':0, 'minkowski_encode':0, 'braycurtis_encode':0,
                        'skew_q1vec_encode':0, 'skew_q2vec_encode':0, 'kur_q1vec_encode':0,
                        'kur_q2vec_encode':0}, index=[0])

In [0]:
def compute_ft_in_batch_tr(start_index, end_index, ft_train):
  
  data = train_data.iloc[start_index:end_index, :]
  
  time_str = get_time()
  print('sentance 2 vector for sentances {} to {} started at '.format(start_index, end_index) + time_str)
  start = time.time()
  with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    v_q1 = session.run(embed(data['question1'].tolist()))
  end = time.time()
  print('It took {} s to embedding {} q1 sentences'.format(end - start, data.shape[0]))

  start = time.time()
  with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    v_q2 = session.run(embed(data['question2'].tolist()))
  end = time.time()
  print('It took {} s to embedding {} q2 sentences'.format(end - start, data.shape[0]))
  time_str = get_time()
  print('sentance 2 vector for sentances {} to {} ended at '.format(start_index, end_index) + time_str)
  
  data = pd.DataFrame()
  data['encode_cos'] = [cosine(x, y) for (x, y) in zip(np.nan_to_num(v_q1),np.nan_to_num(v_q2))]
  data['encode_L1'] = [cityblock(x, y) for (x, y) in zip(np.nan_to_num(v_q1),np.nan_to_num(v_q2))]
  data['encode_L2'] = [euclidean(x, y) for (x, y) in zip(np.nan_to_num(v_q1),np.nan_to_num(v_q2))]
  data['canberra_encode'] = [canberra(x, y) for (x, y) in zip(np.nan_to_num(v_q1),
                                                           np.nan_to_num(v_q2))]
  
  data['minkowski_encode'] = [minkowski(x, y, 3) for (x, y) in zip(np.nan_to_num(v_q1),
                                                            np.nan_to_num(v_q2))]

  data['braycurtis_encode'] = [braycurtis(x, y) for (x, y) in zip(np.nan_to_num(v_q1),
                                                            np.nan_to_num(v_q2))]

  data['skew_q1vec_encode'] = [skew(x) for x in np.nan_to_num(v_q1)]
  data['skew_q2vec_encode'] = [skew(x) for x in np.nan_to_num(v_q2)]
  data['kur_q1vec_encode'] = [kurtosis(x) for x in np.nan_to_num(v_q1)]
  data['kur_q2vec_encode'] = [kurtosis(x) for x in np.nan_to_num(v_q2)]
  
  return ft_train.append(data, ignore_index=True )

In [16]:
for i in range(0, batch_num + 1):
  start_index = i*batch_size
  end_index = (i+1)*batch_size
  if end_index > train_data.shape[0]:
    end_index = train_data.shape[0]
  ft_train = compute_ft_in_batch_tr(start_index, end_index, ft_train)

sentance 2 vector for sentances 0 to 300000 started at 17:34:6
It took 320.63909673690796 s to embedding 300000 q1 sentences
It took 394.0810537338257 s to embedding 300000 q2 sentences
sentance 2 vector for sentances 0 to 300000 ended at 17:46:1
sentance 2 vector for sentances 300000 to 404290 started at 17:50:59
It took 123.26228332519531 s to embedding 104290 q1 sentences
It took 147.40111255645752 s to embedding 104290 q2 sentences
sentance 2 vector for sentances 300000 to 404290 ended at 17:55:30


In [17]:
ft_train.head()

,encode_cos,encode_L1,encode_L2,canberra_encode,minkowski_encode,braycurtis_encode,skew_q1vec_encode,skew_q2vec_encode,kur_q1vec_encode,kur_q2vec_encode
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.036198,4.553006,0.269064,124.261119,0.118751,0.120247,-0.076135,-0.042752,-0.989944,-0.958032
2,0.262635,12.591572,0.724755,221.270339,0.309102,0.347132,-0.056110,-0.026842,-0.942674,-1.278017
3,0.104333,7.645729,0.456800,163.702191,0.198237,0.202258,-0.035711,0.067302,-1.103482,-1.130741
4,0.766393,22.252520,1.238057,325.741225,0.507923,0.757637,-0.073181,-0.100329,-0.788452,-0.822424


In [23]:
ft_train2 = ft_train.iloc[1:ft_train.shape[0],:]
ft_train2.shape

(404290, 10)

In [0]:
ft_train2.reset_index(inplace=True)
ft_train2 = ft_train2.drop('index', axis=1)

In [25]:
ft_train2.head()

,encode_cos,encode_L1,encode_L2,canberra_encode,minkowski_encode,braycurtis_encode,skew_q1vec_encode,skew_q2vec_encode,kur_q1vec_encode,kur_q2vec_encode
0,0.036198,4.553006,0.269064,124.261119,0.118751,0.120247,-0.076135,-0.042752,-0.989944,-0.958032
1,0.262635,12.591572,0.724755,221.270339,0.309102,0.347132,-0.056110,-0.026842,-0.942674,-1.278017
2,0.104333,7.645729,0.456800,163.702191,0.198237,0.202258,-0.035711,0.067302,-1.103482,-1.130741
3,0.766393,22.252520,1.238057,325.741225,0.507923,0.757637,-0.073181,-0.100329,-0.788452,-0.822424
4,0.474255,16.964287,0.973915,262.853599,0.409228,0.504699,-0.018119,-0.107851,-1.244781,-0.965395


In [26]:
ft_train2.to_csv('train_ft_encode.csv', index=False)
upload_to_GCS('train_ft_encode.csv')

train_ft_encode.csv uploaded


In [27]:
# download_from_GCS('train.csv')
download_from_GCS('test.csv')
# train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

test.csv downloaded


In [0]:
test_data = test_data.fillna(" ")

batch_size = 300000
batch_num = test_data.shape[0]//batch_size

ft_test = pd.DataFrame({'encode_cos':0,'encode_L1':0,'encode_L2':0,
                         'canberra_encode':0, 'minkowski_encode':0, 'braycurtis_encode':0,
                        'skew_q1vec_encode':0, 'skew_q2vec_encode':0, 'kur_q1vec_encode':0,
                        'kur_q2vec_encode':0}, index=[0])

In [0]:
def compute_ft_in_batch_ts(start_index, end_index, ft_test):
  
  data = test_data.iloc[start_index:end_index, :]
  
  time_str = get_time()
  print('sentance 2 vector for sentances {} to {} started at '.format(start_index, end_index) + time_str)
  start = time.time()
  with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    v_q1 = session.run(embed(data['question1'].tolist()))
  end = time.time()
  print('It took {} s to embedding {} q1 sentences'.format(end - start, data.shape[0]))

  start = time.time()
  with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    v_q2 = session.run(embed(data['question2'].tolist()))
  end = time.time()
  print('It took {} s to embedding {} q2 sentences'.format(end - start, data.shape[0]))
  time_str = get_time()
  print('sentance 2 vector for sentances {} to {} ended at '.format(start_index, end_index) + time_str)
  
  data = pd.DataFrame()
  data['encode_cos'] = [cosine(x, y) for (x, y) in zip(np.nan_to_num(v_q1),np.nan_to_num(v_q2))]
  data['encode_L1'] = [cityblock(x, y) for (x, y) in zip(np.nan_to_num(v_q1),np.nan_to_num(v_q2))]
  data['encode_L2'] = [euclidean(x, y) for (x, y) in zip(np.nan_to_num(v_q1),np.nan_to_num(v_q2))]
  data['canberra_encode'] = [canberra(x, y) for (x, y) in zip(np.nan_to_num(v_q1),
                                                           np.nan_to_num(v_q2))]
  
  data['minkowski_encode'] = [minkowski(x, y, 3) for (x, y) in zip(np.nan_to_num(v_q1),
                                                            np.nan_to_num(v_q2))]

  data['braycurtis_encode'] = [braycurtis(x, y) for (x, y) in zip(np.nan_to_num(v_q1),
                                                            np.nan_to_num(v_q2))]

  data['skew_q1vec_encode'] = [skew(x) for x in np.nan_to_num(v_q1)]
  data['skew_q2vec_encode'] = [skew(x) for x in np.nan_to_num(v_q2)]
  data['kur_q1vec_encode'] = [kurtosis(x) for x in np.nan_to_num(v_q1)]
  data['kur_q2vec_encode'] = [kurtosis(x) for x in np.nan_to_num(v_q2)]
  
  return ft_test.append(data, ignore_index=True )

In [30]:
for i in range(0, batch_num + 1):
  start_index = i*batch_size
  end_index = (i+1)*batch_size
  if end_index > test_data.shape[0]:
    end_index = test_data.shape[0]
  ft_test = compute_ft_in_batch_ts(start_index, end_index, ft_test)

sentance 2 vector for sentances 0 to 300000 started at 18:36:18
It took 329.34953904151917 s to embedding 300000 q1 sentences
It took 340.0029604434967 s to embedding 300000 q2 sentences
sentance 2 vector for sentances 0 to 300000 ended at 18:47:28
sentance 2 vector for sentances 300000 to 600000 started at 18:52:19
It took 342.4445502758026 s to embedding 300000 q1 sentences
It took 325.1744349002838 s to embedding 300000 q2 sentences
sentance 2 vector for sentances 300000 to 600000 ended at 19:3:27
sentance 2 vector for sentances 600000 to 900000 started at 19:8:19
It took 410.13166546821594 s to embedding 300000 q1 sentences
It took 408.4795072078705 s to embedding 300000 q2 sentences
sentance 2 vector for sentances 600000 to 900000 ended at 19:21:58
sentance 2 vector for sentances 900000 to 1200000 started at 19:26:51
It took 416.1342251300812 s to embedding 300000 q1 sentences
It took 337.99826860427856 s to embedding 300000 q2 sentences
sentance 2 vector for sentances 900000 to 1

In [31]:
ft_test.head()

,encode_cos,encode_L1,encode_L2,canberra_encode,minkowski_encode,braycurtis_encode,skew_q1vec_encode,skew_q2vec_encode,kur_q1vec_encode,kur_q2vec_encode
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.307394,13.228489,0.784084,230.459080,0.335352,0.373265,-0.009512,0.000340,-1.244313,-1.233780
2,0.119225,8.212183,0.488314,167.320022,0.209699,0.216852,0.043952,0.034034,-1.034802,-1.156297
3,0.149012,9.238203,0.545916,186.358232,0.234621,0.254429,0.005193,0.023300,-0.798762,-0.700285
4,0.284970,13.437407,0.754944,248.603338,0.315308,0.387871,0.054974,0.111906,-0.414613,-1.089561


In [32]:
ft_test2 = ft_test.iloc[1:ft_test.shape[0],:]
ft_test2.reset_index(inplace=True)
ft_test2 = ft_test2.drop('index', axis=1)
ft_test2.shape

(2345796, 10)

In [33]:
ft_test2.head()

,encode_cos,encode_L1,encode_L2,canberra_encode,minkowski_encode,braycurtis_encode,skew_q1vec_encode,skew_q2vec_encode,kur_q1vec_encode,kur_q2vec_encode
0,0.307394,13.228489,0.784084,230.459080,0.335352,0.373265,-0.009512,0.000340,-1.244313,-1.233780
1,0.119225,8.212183,0.488314,167.320022,0.209699,0.216852,0.043952,0.034034,-1.034802,-1.156297
2,0.149012,9.238203,0.545916,186.358232,0.234621,0.254429,0.005193,0.023300,-0.798762,-0.700285
3,0.284970,13.437407,0.754944,248.603338,0.315308,0.387871,0.054974,0.111906,-0.414613,-1.089561
4,0.401571,15.591051,0.896182,269.073492,0.381538,0.475944,0.020180,-0.040126,-0.705323,-0.276419


In [34]:
ft_test2.to_csv('test_ft_encode.csv', index=False)
upload_to_GCS('test_ft_encode.csv')

test_ft_encode.csv uploaded


In [0]:
train_data = train_data.fillna("")

start = time.time()
with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  q1_emb_train = session.run(embed(train_data['question1'].tolist()))
end = time.time()
print('It took {} s to embedding {} q1 sentences'.format(end - start, train_data.shape[0]))
print(q1_emb_train.shape)

start = time.time()
with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  q2_emb_train = session.run(embed(train_data['question2'].tolist()))
end = time.time()
print('It took {} s to embedding {} q2 sentences'.format(end - start, train_data.shape[0]))
print(q2_emb_train.shape)

start = time.time()

L2_train = np.sqrt(np.square(q1_emb_train - q2_emb_train ).sum(axis=1))

end = time.time()
print('It took {} s to compute L2 for {} sentences'.format(end - start, train_data.shape[0]))

print(L2_train.shape)

It took 347.80715894699097 s to embedding 404290 q1 sentences
(404290, 512)
It took 390.8409321308136 s to embedding 404290 q2 sentences
(404290, 512)
It took 1.1863055229187012 s to compute L2 for 404290 sentences
(404290,)


In [0]:
L1_train = np.sqrt(np.abs(q1_emb_train - q2_emb_train ).sum(axis=1))
print(L1_train.shape)

(404290,)


In [0]:
norm_q1 = np.square(q1_emb_train).sum(axis=1)**0.5
print(norm_q1.shape)

(404290,)


In [0]:
norm_q2 = np.square(q2_emb_train).sum(axis=1)**0.5
print(norm_q2.shape)

(404290,)


In [0]:
cos_train = np.multiply(q1_emb_train, q2_emb_train).sum(axis=1) / (norm_q1 * norm_q2)
print(cos_train.shape)

(404290,)


In [0]:
ft_train = pd.DataFrame()
ft_train['encode_L1'] = L1_train
ft_train['encode_L2'] = L2_train
ft_train['encode_cos'] = cos_train

In [0]:
ft_train.head()

,encode_L1,encode_L2,encode_cos
0,2.133778,0.269064,0.963802
1,3.548460,0.724755,0.737365
2,2.765091,0.456800,0.895667
3,4.717258,1.238057,0.233607
4,4.118772,0.973915,0.525745


In [0]:
ft_train.to_csv('encode_ft_train.csv', index=False)

In [0]:
from googleapiclient.http import MediaFileUpload
def upload_to_GCS(file_name): 
  media = MediaFileUpload(file_name, 
                          mimetype='text/csv',
                          resumable=True)

  request = gcs_service.objects().insert(bucket=bucket_name, 
                                         name=file_name,
                                         media_body=media)

  response = None
  while response is None:
    # _ is a placeholder for a progress object that we ignore.
    # (Our file is small, so we skip reporting progress.)
    _, response = request.next_chunk()
  print(file_name + ' uploaded')

In [0]:
upload_to_GCS('encode_ft_train.csv')

encode_ft_train.csv uploaded


In [0]:
del q1_emb_train, q2_emb_train, L1_train, L2_train, norm_q2, norm_q1, cos_train, ft_train

In [0]:
test_data.shape[0]

2345796

In [0]:
a = 123
b =10
n = a//b

for i in range(0, n + 1):
  start_index = i*b
  end_index = (i+1)*b -1
  if end_index > a:
    end_index = a -1
  print(start_index, end_index)

0 9
10 19
20 29
30 39
40 49
50 59
60 69
70 79
80 89
90 99
100 109
110 119
120 122


In [0]:
ft_test = pd.DataFrame({'encode_L1':0, 'encode_L2':0, 'encode_cos':0}, index=[0])

In [0]:

def compute_ft_in_batch(start_index, end_index, ft_test):
  start = time.time()
  with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    q1_emb_test = session.run(embed(test_data['question1'][start_index:end_index].tolist()))
  end = time.time()
  print('It took {} s to embedding {} q1 sentences'.format(end - start, end_index - start_index + 1))
  print(q1_emb_test.shape)

  start = time.time()
  with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    q2_emb_test = session.run(embed(test_data['question2'][start_index:end_index].tolist()))
  end = time.time()
  print('It took {} s to embedding {} q2 sentences'.format(end - start, end_index - start_index + 1))
  print(q2_emb_test.shape)

  start = time.time()

  L2_test = np.sqrt(np.square(q1_emb_test - q2_emb_test ).sum(axis=1))

  end = time.time()
  print('It took {} s to compute L2 for {} sentences'.format(end - start, end_index - start_index + 1))

  print(L2_test.shape)

  L1_test = np.sqrt(np.abs(q1_emb_test - q2_emb_test ).sum(axis=1))
  print(L1_test.shape)

  norm_q1 = np.square(q1_emb_test).sum(axis=1)**0.5
  print(norm_q1.shape)

  norm_q2 = np.square(q2_emb_test).sum(axis=1)**0.5
  print(norm_q2.shape)

  cos_test = np.multiply(q1_emb_test, q2_emb_test).sum(axis=1) / (norm_q1 * norm_q2)
  print(cos_test.shape)


  ft = pd.DataFrame()
  ft['encode_L1'] = L1_test
  ft['encode_L2'] = L2_test
  ft['encode_cos'] = cos_test
  return ft_test.append(ft, ignore_index=True )


In [0]:
test_data = test_data.fillna("")


batch_size = 400000
batch_num = test_data.shape[0]//batch_size


for i in range(0, batch_num + 1):
  start_index = i*batch_size
  end_index = (i+1)*batch_size -1
  if end_index > test_data.shape[0]:
    end_index = test_data.shape[0] -1
  ft_test = compute_ft_in_batch(start_index, end_index, ft_test)

It took 347.03724241256714 s to embedding 400000 q1 sentences
(399999, 512)
It took 345.68520307540894 s to embedding 400000 q2 sentences
(399999, 512)
It took 0.6193387508392334 s to compute L2 for 400000 sentences
(399999,)
(399999,)
(399999,)
(399999,)
(399999,)
It took 376.66708850860596 s to embedding 400000 q1 sentences
(399999, 512)
It took 387.702068567276 s to embedding 400000 q2 sentences
(399999, 512)
It took 0.6943178176879883 s to compute L2 for 400000 sentences
(399999,)
(399999,)
(399999,)
(399999,)
(399999,)
It took 389.8580102920532 s to embedding 400000 q1 sentences
(399999, 512)
It took 388.38801407814026 s to embedding 400000 q2 sentences
(399999, 512)
It took 0.7225923538208008 s to compute L2 for 400000 sentences
(399999,)
(399999,)
(399999,)
(399999,)
(399999,)
It took 382.55007338523865 s to embedding 400000 q1 sentences
(399999, 512)
It took 381.72921538352966 s to embedding 400000 q2 sentences
(399999, 512)
It took 0.722426176071167 s to compute L2 for 400000 

In [0]:
ft_test.head()

,encode_L1,encode_L2,encode_cos
0,0.000000,0.000000,0.000000
1,3.637099,0.784084,0.692606
2,2.865691,0.488314,0.880775
3,3.039441,0.545916,0.850988
4,3.665707,0.754944,0.715030


In [0]:
ft_test.shape

(2345791, 3)

In [0]:
ft_test_trim = ft_test.iloc[1:ft_test.shape[0],:]

In [0]:
ft_test_trim.head()

,encode_L1,encode_L2,encode_cos
1,3.637099,0.784084,0.692606
2,2.865691,0.488314,0.880775
3,3.039441,0.545916,0.850988
4,3.665707,0.754944,0.715030
5,3.948551,0.896182,0.598429


In [0]:
ft_test_trim.reset_index(inplace=True)
ft_test_trim.head()

,index,encode_L1,encode_L2,encode_cos
0,1,3.637099,0.784084,0.692606
1,2,2.865691,0.488314,0.880775
2,3,3.039441,0.545916,0.850988
3,4,3.665707,0.754944,0.715030
4,5,3.948551,0.896182,0.598429


In [0]:
ft_test_trim2 = ft_test_trim.iloc[:,1:4]
ft_test_trim2.head()

,encode_L1,encode_L2,encode_cos
0,3.637099,0.784084,0.692606
1,2.865691,0.488314,0.880775
2,3.039441,0.545916,0.850988
3,3.665707,0.754944,0.715030
4,3.948551,0.896182,0.598429


In [0]:
ft_test_trim2.to_csv('encode_ft_test.csv')
upload_to_GCS('encode_ft_test.csv')

encode_ft_test.csv uploaded
